# <center>Machine learning from scratch - Part II</center>
## <center>EMBO practical course on population genomics 2019 @ Procida, Italy</center>
### <center>Marco Chierici & Margherita Francescatto</center>
#### <center>_FBK/MPBA_</center>

Recap. We are using a subset of the SEQC neuroblastoma data set [Zhang et al, Genome Biology, 2015] consisting of 272 samples (136 training, 136 test). The data was preprocessed a bit to facilitate the progress of the tutorial.

We start by loading the modules we need to process the data.

In [1]:
import numpy as np
import pylab as pl ## for plotting
import pandas as pd ## for reading text files and manipulating data frames
from sklearn import neighbors ## kNN classifier
from sklearn import svm ## SVM classifier
from sklearn.ensemble import RandomForestClassifier ## RF classifier
from sklearn.model_selection import cross_val_score ## needed to train in CV
%matplotlib inline
np.random.seed(42) ## set random seed just in case

Define files to read:

In [5]:
##  for convenience, define the data directory as a variable
DATA_DIR = "NB_data/" #"/data/marco/"

In [6]:
DATA_TR = DATA_DIR + "MAV-G_272_tr.txt"
DATA_TS = DATA_DIR + "MAV-G_272_ts.txt"
LABS_TR = DATA_DIR + "labels_tr.txt"
LABS_TS = DATA_DIR + "labels_ts.txt"

Read in the files as _pandas dataframes_ (they are conceptually like R data frames):

In [7]:
data_tr = pd.read_csv(DATA_TR, sep = "\t")
data_ts = pd.read_csv(DATA_TS, sep = "\t")

Since we already looked at the data in the first part of the dataset, we move directly to the juicy stuff.

We drop the first column from the train and test expression sets, since it's just the sample IDs...

In [8]:
data_tr = data_tr.drop('sampleID',axis =1)
data_ts = data_ts.drop('sampleID',axis =1)

...and store the data into Numpy arrays.

In [9]:
x_tr = data_tr.values
x_ts = data_ts.values

Now we read in the files containing labels and select the column with the CLASS target to do our first round of analyses.

In [10]:
labs_tr = pd.read_csv(LABS_TR, sep = "\t")
labs_ts = pd.read_csv(LABS_TS, sep = "\t")
class_lab_tr = labs_tr[['CLASS']]
class_lab_ts = labs_ts[['CLASS']]
y_tr = class_lab_tr.values.ravel()
y_ts = class_lab_ts.values.ravel()

In the previous part of the tutorial, we focused on the k-NN classifiers. In the previous lecture, however, we explored theoretical aspects related to two other broadly used classifiers: Support Vector Machines (SVMs) and Random Forests (RFs). In this second part of tutorial, the first thing we want to do is assessing how these two alternative classification methods perform on our neuroblastoma dataset.

We start with SVM. We first rescale the data, import the relevant model and create an instance of the SVM classifier.

In [11]:
from sklearn.preprocessing import MinMaxScaler
## first you need to create a "scaler" object
scaler = MinMaxScaler(feature_range=(-1,1))
## then you actually scale data by fitting the scaler object on the data
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)

In [12]:
## import support vector classifier (SVC) and create an instance
from sklearn.svm import SVC
svc = SVC(random_state=42, verbose=1, kernel='linear')

Note that the specification _kernel = 'linear'_ implies that a linear kernel will be used. If you remember from the lecture, this means that a linear function is used to define the decision boundaries of the classifier. Alternatives include _‘poly’_ and _‘rbf’_ for polynomial or gaussian kernels respectively. Scikit-learn offers an alternative implementation of linear SVMs. You can find more details in Scikit User Guide. 

As previously done with the k-NN classifier, we fit the SVM and get the predictions for the test data.

In [13]:
## fit the model and get the predictions
svc.fit(x_tr, y_tr)
class_pred_ts = svc.predict(x_ts)

[LibSVM]

Now we give a look at the classification metrics introduced in the first part of the tutorial. to access the functions, we need to load the metrics module.

In [14]:
from sklearn import metrics
print('MCC = ', metrics.matthews_corrcoef(class_lab_ts, class_pred_ts))
print('ACC = ', metrics.accuracy_score(class_lab_ts, class_pred_ts))
print('SENS = ', metrics.recall_score(class_lab_ts, class_pred_ts))

MCC =  0.8857501367027195
ACC =  0.9485294117647058
SENS =  0.9555555555555556


We can also give a look at the classification report.

In [15]:
print(metrics.classification_report(class_lab_ts, class_pred_ts))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92        46
           1       0.97      0.96      0.96        90

   micro avg       0.95      0.95      0.95       136
   macro avg       0.94      0.95      0.94       136
weighted avg       0.95      0.95      0.95       136



Exercise: **one-shot Random Forest classification**. _Hint:_ the RF classifier is implemented in the Scikit learn class RandomForestClassifier, from _sklearn.ensemble_ module.

In [25]:
## space for exercise

from sklearn.ensemble import RandomForestClassifier as RFC
clf = RFC(n_estimators=500)
clf.fit(x_tr, y_tr)
y_pred_rfc = clf.predict(x_ts)
print('MCC = ', metrics.matthews_corrcoef(class_lab_ts, y_pred_rfc))
print('ACC = ', metrics.accuracy_score(class_lab_ts, y_pred_rfc))
print('SENS = ', metrics.recall_score(class_lab_ts, y_pred_rfc))
print(metrics.classification_report(class_lab_ts, y_pred_rfc))

## Parameter tuning

As mentioned in the lecture, Scikit learn offers a very useful and flexible tool for parameter tuning called _GridSearchCV_. While the tool is very sophisticated and efficient, it is useful to at least try an example _by hand_ to understand what is happening in the background.

For this example we use a linear SVM and try to tune the C parameter. You might remember from the lectures that the paramenter C essentially controls how much we want to avoid misclassifying each training example. Large values of C result in smaller margins, i.e. closer fitting to the training data. As mentioned in the classes, the drawback is over-fitting, resulting in poor generalization.

In [ ]:
## define the sequence of C values we want to use in the search of the best one
C_list = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
for C in C_list:
    print('C = ', C)
    svc = svm.SVC(kernel = 'linear', C=C)
    svc.fit(x_tr, class_lab_tr.values.ravel())
    class_pred_ts = svc.predict(x_ts)
    print('MCC = ', metrics.matthews_corrcoef(class_lab_ts, class_pred_ts))
    print('ACC = ', metrics.accuracy_score(class_lab_ts, class_pred_ts))
    print('SENS = ', metrics.recall_score(class_lab_ts, class_pred_ts), "\n")

From C = 1e-03 the classification performance reaches a plateau. C = 1e-04 yields the highest MCC on the test set: when tuning the parameters we would consider this as the best choice for the problem.

**Exercise:** as you already saw in the lectures, there are many parameters that can be tuned, also when considering only one simple classifier. For example, if you consider SVM with 'rbf' kernel, you could check performance changes with different values of C **and** gamma, for example using two nested loops.

In [17]:
## space for exercise

As we mentioned, Scikit offers fully automated parameter tuning engine. We illustrate its power with a simple example on our data. We use GridSearchCV to search through a grid of C and gamma parameter options for SVM with 'rbf' kernel. In order to do this we define a small function svc_param_selection that does the work for us.

In [18]:
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds, n_jobs=4)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

svc_param_selection(x_tr, y_tr, 5)

{'C': 0.001, 'gamma': 0.001}

## Feature ranking

As mentioned in the lecture, random forests have a built in tool for feature ranking

In [19]:
# Build a forest and compute the feature importances
rf = RandomForestClassifier(n_estimators=250)
rf.fit(x_tr, y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

For the sake of completeness make the predictions and check the classification performance.

In [20]:
class_pred_ts = rf.predict(x_ts)
print('MCC = ', metrics.matthews_corrcoef(class_lab_ts, class_pred_ts))
print('ACC = ', metrics.accuracy_score(class_lab_ts, class_pred_ts))
print('SENS = ', metrics.recall_score(class_lab_ts, class_pred_ts))

MCC =  0.8704408378703687
ACC =  0.9411764705882353
SENS =  0.9444444444444444


Now extract the feature importances and display the first 10:

In [21]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking (top 10 features):")
for f in range(10):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking (top 10 features):
1. feature 5992 (0.011946)
2. feature 10739 (0.011095)
3. feature 7384 (0.010125)
4. feature 4365 (0.009361)
5. feature 9254 (0.008915)
6. feature 17952 (0.007227)
7. feature 10084 (0.006966)
8. feature 1442 (0.006885)
9. feature 21809 (0.006721)
10. feature 5892 (0.006540)


Would be nice to know to which genes they actually correspond. If you remember the gene names are the column names of the pandas dataframe containing the training/test data.

In [22]:
columnsNamesArr = data_tr.columns.values
for i in range(10):
    print(columnsNamesArr[indices[i]])

MRPL11.Gene_AceView
POLA2.Gene_RefSeq
UHRF1.Gene_AceView
C14orf166.Gene_AceView
GMPS.Gene_AceView
ERCC6L.Gene_AceView
MAMLD1.Gene_AceView
PHLDB1.Gene_AceView
snawjarby.Gene_AceView
MAP7.Gene_AceView


## Extra exercises

The classification task considered so far (CLASS) is quite easy, since the classes reflect extreme disease outcomes (favorable vs unfavorable).

A more interesting task could be the prediction of Event-Free Survival (EFS). To do this, an extended version of the dataset is provided in the `/data/marco` directory:

In [ ]:
DATA_TR = DATA_DIR + "full_MAV-G_272_tr.txt"
DATA_TS = DATA_DIR + "full_MAV-G_272_ts.txt"
LABS_TR = DATA_DIR + "full_labels_tr.txt"
LABS_TS = DATA_DIR + "full_labels_ts.txt"

Read the data in and prepare the `x_tr`, `x_ts`,  `y_tr`, `y_ts` Numpy arrays, as before, using "EFS" as target variable.

Recalling concepts from the first practical, perform an explorative PCA analyisis, plotting the first two components.

Train a kNN classifier in one-shot mode: fit the model on the training set and predict the labels on the test set. Compute performance metrics using the provided true labels of the test set.

Experiment with different classifier(s) and/or different parameters.

Try tuning the parameters (e.g. using GridSearchCV) and find the optimal parameter set.

Using the optimal parameters, run a (iterated) cross-validation on the training set; compute the average cross-validation metrics.

Using the optimal parameters, train a model on the whole training set and predict the labels of the test set. Compute the metrics and compare them with the average cross-validation metrics. What do you expect? Use the trained model to rank the features and inspect the top ones.